## Introduction

An Ising spin model can be thought of as a graph with vertices $V =$ {$ v_0, v_1, v_2, ..., v_{n-1}$} and edges, in which each vertex has a spin which can be positive (+1) or negative (-1). The spin of vertex is denoted as $s_v$. The vertices have weights given by a vector $h$ and the edges have weights given by an upper triangular matrix $J$; weights can be positive, negative, or zero. The spin configuration, i.e., whether each vertex's spin is positive or negative, is known as the _state_. The energy of the system depends on the state and the weights given by $h$ and $J$. The equation for the system energy given by state $S =$ {$s_0, s_1, s_2, ..., s_{n-1}$} is:

$$\begin{align}
E(s_0, s_1, s_2, ..., s_{n-1}) = \sum_{0 \leq i \leq {n-1}} h_i s_i + \sum_{0 \leq i < j \leq {n-1}} J_{ij} s_i s_j \\
\label{eq:energy} \tag{1}
\end{align}$$

We are interested in finding a *ground state*, i.e. a state that minimizes the energy of the system.

## Algorithm Description
A ground state (optimal solution) $S^* =$ {$s_0^*, s_1^*, s_2^*, ..., s_{n-1}^*$} can be found using _dynamic programming_ as follows. Let $v_r \in V$ be an arbitrary chosen root vertex in the graph (the choice does not affect the solution). From this root, each vertex $v_i \in V$ has a depth $d_i$ which is the number of edges between it and $v_r$ (the depth of $v_r$ is zero). The children, $C_i$, of vertex $v_i$ are those neighboring (connected) vertices, if any, of depth $d_i + 1$. Every vertex $v_i$ other than the root has a __unique__ parent, which is the neighboring vertex of depth $d_i - 1$. 

The figure below illustrates these concepts through an example, where the root of a tree is labeled $r$ and every other node is labeled with its depth. The dashed vertices are the descendents of the bold vertex. The best labels for the dashed verticies can be computed as a function of the label of the bold vertex.
![Fig1](Fig1.png)

For any vertex $v_j \in V$ with no children (i.e. any leaf of the rooted tree), the best spin $s_j^*$ for that vertex can be computed as a function of the spin of just its parent $v_i$, denoted by $\sigma$. The only edge incident on $v_j$ is $(v_i, v_j)$, thus the only contribution of $s_j$ to the energy in (1) is $h_j s_j + J_{ij} s_j \sigma$. The quality of the best spin for $v_j$ given spin $\sigma$ for $v_i$ is

$$\begin{align} 
B_j(\sigma) = \min_{s_j} \ \bigl(h_j s_j + J_{ij} s_j \sigma\bigr) .\\ 
\label{eq:Bleaf} \tag{2} 
\end{align}$$

\begin{equation*}
\mathbf{r} \equiv \begin{bmatrix}
y \\
\theta
\end{bmatrix}
\label{eq:vector_ray} \tag{1}
\end{equation*}

Vector **r** is defined by equation $\eqref{eq:vector_ray}$

For any vertex $v_j \in V$ other than the root, assume that the function $B_c(s_j)$ is known for each child $v_c \in C_j$. That is, the quality of the best spin for each child is known with respect to the spin of $v_j$. Then the quality of the best spin for $v_j$ given spin $\sigma$ for its parent $v_i$ is

$$\begin{align}\
B_j(\sigma) = \min_{s_j} \ \biggl(h_j s_j + J_{ij} s_j \sigma + \sum_{v_c \in C_j} B_c(s_j)\biggr).\\
\label{eq:B_general} \tag{3}
\end{align}$$

This equation subsumes (2) because for a leaf node the sum over its children is simply empty. Finally, for the root $v_r$, if $B_c(s_r)$ is known for each child $v_c \in C_r$ then the best spin for the root is

$$\begin{align}\
s_r^* = \text{arg} \min_{s_r} \ \biggl(h_r s_r + \sum_{v_c \in C_r} B_c(s_r)\biggr).\\
\label{eq:B_root} \tag{4}
\end{align}$$

That is, the minimization in (1) can be expressed recursively in terms of the $(n-1)$ functions $B_j(\sigma)$ for each vertex $v_j \in V$ (other than the root). At this point, we compute the optimal spin $s_r^*$ for the root. The optimal state $S^*$ for all the remaining nodes can be computed by tracing back from the root to each leaf. We know the optimal spin of each node given the spin of its parent, and the optimal spin of each parent is now known starting from the root. That is, the best spin for $v_j$ as a function of $\sigma^*$, the known optimal spin of its parent, can be obtained by replacing $min$ in (3) with $argmin$:

$$\begin{align}\
s_j^* = \text{arg} \min_{s_j} \ \biggl(h_j s_j + J_{ij} s_j \sigma^* + \sum_{v_c \in C_j} B_c(s_j)\biggr).\\
\label{eq:opt_spin} \tag{5}
\end{align}$$

## alaky

$$\begin{align}
E(s_0, s_1, s_2, ..., s_{n-1}) = \sum_{0 \leq i \leq {n-1}} h_i s_i + \sum_{0 \leq i < j \leq {n-1}} J_{ij} s_i s_j \\
\label{eq:energy} \tag{1}
\end{align}$$

## Input

In [1]:
import numpy as np

# read the input file and save all lines in a list
lines = []
with open("input.txt", "rt") as input_file:
    for line in input_file:  
        if not line.strip(): continue  # skip empty lines  
        
        # remove all whitespace characters (space, tab, newline, return, formfeed)
        # before, inside (just keep one space), and after each line before adding to the list 'lines'            
        lines.append(" ".join(line.split()))  
 
lines

['c This is an extra-hard problem',
 'p test01 4 6',
 '0 1 1',
 '1 2 1',
 'c this is a test comment',
 '1 3 1',
 '0 0 -1',
 '1 1 -1',
 '2 2 -1']

In [2]:
# find the index of problem line which starts with lower case p
for i, line in enumerate(lines):
    if line[0] == "p":
        p_index = i   
        break
p_index

1

In [3]:
# find the number of vertices and the number of (non-zero) weights from the problem line
num = [int(i) for i in lines[p_index].split() if i.isdigit()] 
num

[4, 6]

In [4]:
n = num[0]  # number of vertices in the graph

###### Assumption: the index of vertices start from 0 not 1! 
$V =$ {$ v_0, v_1, v_2, ..., v_{n-1}$}

In [5]:
# initiate all weights in h and j as zero
def weights(n): 
    h = {i: 0 for i in range(n)}
    J = {(i, j): 0 for i in range(n) for j in range(n)}
    
    return h, J

In [6]:
# read the data lines and save the weights in h and J
h, J = weights(n)
num_data_lines = 0

for i in range(p_index+1, len(lines)):  # data lines appear after the problem line
    if lines[i][0].isdigit():           # this is a data line
        
        num_data_lines += 1
        
        u = int(lines[i].split()[0])
        v = int(lines[i].split()[1])
        w = int(lines[i].split()[2])
        if u != v:                     # the (J) weight is assigned to the edge between nodes u and v
            J[(u, v)] = J[(v, u)] = w  # J is an upper triangular matrix
        else:
            h[u] = w                   # the (h) weight is assigned to the node
            
if num_data_lines == num[1]:
    print('The number of weights given in problem line is equal to the number of data lines in the file.')
else:
    raise Exception('ERROR: The number of weights given in problem line is NOT equal to the number of data lines in the file!')

The number of weights given in problem line is equal to the number of data lines in the file.


## Algorithm Implementation

In [7]:
# select root vertex randomly
r = np.random.randint(0, n)  # assumption: the index of vertices start from 0 not 1!
print('Root is randomly selected as vertex {}.'.format(r))

Root is randomly selected as vertex 2.


In [8]:
d = {i: np.nan for i in range(n)}  # depth: number of edges between vertex i and root
d[r] = 0  # depth of root is zero

VV = [i for i in range(n)]  # variable (decreasing) set of vertices

crt_d = 0  # current depth

# parent: every vertex i (other than the root) has a unique parent, which is the neighboring vertex of depth 𝑑𝑖−1
prt = {i: np.nan for i in range(n)}

# children of vertex i are the nodes connected to i, which have depth 𝑑𝑖+1
chd = {i: [] for i in range(n)}

while len(VV) > 0: 
    
    # fid all vertices with the current depth
    V_crt_d = []  
    for v in VV:
        if d[v] == crt_d:
            V_crt_d.append(v)
    
    for i in V_crt_d:  # iterate over all vertices with the current depth (parents) and find their children
        for j in VV:
            if j not in V_crt_d:  # two vertices with the same depth cannot be parent or child of each other 
                if J[(i, j)] != 0:
                            
                    # vertex i will be the parent of node j
                    prt[j] = i

                    # node j to be added as a child of vertex i
                    chd[i].append(j)

                    # depth of node j (child) will be the depth of parent + 1
                    d[j] = crt_d + 1

        VV.remove(i)  # remove vertex i from VV; all of its children are found

    crt_d += 1  # go deeper to the next depth 

In [9]:
print('Vertex \t Depth')
for key in d.keys():
    print(key, '\t', d[key])

Vertex 	 Depth
0 	 2
1 	 1
2 	 0
3 	 2


In [10]:
print('Vertex \t Children')
for key in chd.keys():
    print(key, '\t', chd[key])

Vertex 	 Children
0 	 []
1 	 [0, 3]
2 	 [1]
3 	 []


In [11]:
print('Vertex \t Parent')
for key in prt.keys():
    print(key, '\t', prt[key])

Vertex 	 Parent
0 	 1
1 	 2
2 	 nan
3 	 1


The quality of the best spin for $v_j$ given spin $\sigma$ for its parent $v_i$, shown in (3), can be restated by replacing $s_j$ with $\pm 1$:

$$\begin{align}\
B_j(\sigma) = \min \ \biggl(h_j + J_{ij} \sigma + \sum_{v_c \in C_j} B_c(+1) \quad , \quad 
                            - h_j - J_{ij} \sigma + \sum_{v_c \in C_j} B_c(-1)\biggr)\\
\end{align}$$

In [12]:
# vertex i is the parent of vertex j and has spin sigma
# B is a function of sigma
# sigma = +-1

def B(j, sigma): 
    
    BC_p = BC_n = 0
    for c in chd[j]:
        BC_p += B(c, +1)
        BC_n += B(c, -1)
    
    i = prt[j]
    output = min(h[j] + J[i, j]*sigma +  BC_p , -h[j] - J[i, j]*sigma + BC_n)
    
    return output

For the root $v_r$, now $B_c(s_r)$ is known for each child $v_c \in C_r$, so the best spin for the root is calculated using (4):

$$\begin{align}\
s_r^* = \text{arg} \min_{s_r} \ \biggl(h_r s_r + \sum_{v_c \in C_r} B_c(s_r)\biggr) \\
\end{align}$$

In [13]:
def B_sr(sr):
    
    BC = 0
    for c in chd[r]:
        BC += B(c, sr)

    return h[r]*sr + BC


if B_sr(+1) < B_sr(-1):
    opt_sr = +1
else:
    opt_sr = -1
    
print('Optimal spin for the root (vertex {}) = {}.'.format(r, opt_sr))

Optimal spin for the root (vertex 2) = 1.


In [14]:
# optimal spin for each node
opt_s = {i: np.nan for i in range(n)}
opt_s[r] = opt_sr

The best spin for $v_j$ as a function of $\sigma^*$, the known optimal spin of its parent, can be obtained by (5):

$$\begin{align}\
s_j^* = \text{arg} \min_{s_j} \ \biggl(h_j s_j + J_{ij} s_j \sigma^* + \sum_{v_c \in C_j} B_c(s_j)\biggr) \\
\end{align}$$

In [15]:
# vertex i is the parent of vertex j  
# sj = +-1

def B2(j, sj): 
    
    BC = 0
    for c in chd[j]:
        BC += B(c, j)
    
    i = prt[j]
    output = h[j]*sj + J[i, j]*sj*opt_s[i] + BC
    
    return output

def sel_opt_s(j):
    if B2(j, +1) < B2(j, -1):
        opt_s[j] = +1
    else:
        opt_s[j] = -1

The optimal state $S^*$ for the nodes can be computed by tracing back from the root to each leaf, i.e. in order of increasing depth.

In [16]:
# sort the depth values of the nodes in ascending order
import collections
from operator import itemgetter 

od_a = collections.OrderedDict(sorted(d.items(), key=itemgetter(1), reverse=False))
od_a

OrderedDict([(2, 0), (1, 1), (0, 2), (3, 2)])

In [17]:
# start with vertices (other than the root) with lower depth values
for j in od_a.keys():
    if j == r:
        continue   
    sel_opt_s(j)

print('Vertex \t Optimal Spin')
for key in opt_s.keys():
    print(key, '\t', opt_s[key])

Vertex 	 Optimal Spin
0 	 1
1 	 -1
2 	 1
3 	 1


Finally the optimal energy of the system is calculated using (1):

$$\begin{align}\
E^* = \sum_{0 \leq i \leq {n-1}} h_i s_i^* + \sum_{0 \leq i < j \leq {n-1}} J_{ij} s_i^* s_j^* \\
\end{align}$$

In [18]:
opt_e = 0
for i in range(n):
    opt_e += h[i]*opt_s[i]
    for j in range(n):
        if i < j:
            opt_e += J[i,j]*opt_s[i]*opt_s[j]
        
print('Optimal energy of the system = {}.'.format(opt_e))

Optimal energy of the system = -4.


## Output

In [19]:
print(opt_e)

for key in opt_s.keys():
    if opt_s[key] == +1:
        print("+", end=" ")
    else:
        print("-", end=" ")

-4
+ - + + 

In [20]:
# write the output to a text file
with open("output.txt", "w") as output_file:
    print(opt_e, file=output_file)

    for key in opt_s.keys():
        if opt_s[key] == +1:
            print("+", end=" ", file=output_file)
        else:
            print("-", end=" ", file=output_file)

## References
- Felzenszwalb PF, Huttenlocher DP. _Pictorial Structures for Object Recognition_. International Journal of Computer Vision 61, 55-79 (2005) https://doi.org/10.1023/B:VISI.0000042934.15159.49


- Felzenszwalb PF, Zabih R. _Dynamic Programming and Graph Algorithms in Computer Vision_. IEEE Transactions on Pattern Analysis and Machine Intelligence 33, 721-740 (2011) https://doi.org/10.1109/TPAMI.2010.135